# [Method for fitting data to the sum of N exponentials](https://math.stackexchange.com/a/3808325/605948)

Assuming the data set $(y, x)$;

Compute the $n$ integrals of $y$ ; $\int y, \int^2{y}, \cdots , , \int^{n-1}{y} , \int^n{y}$.

Compute the $n-1$ powers of the independent variable $x$ ; $x, x^2, \cdots , x^{n-1}$.

Solve linear least squares problem for $[a_{1}, a_{2}, \cdots, a_{n-1}, a_{n}, k_{n-1}, k_{n-2} , \cdots , k_{2}, k_{1}, k_{0}]$ in:

$$
y = \left[ \int y , \int^2{y} , \cdots, \int^{n-1}{y}, \int^n{y} , x^{n-1}, x^{n-2}, \cdots , x^{2}, x, 1 \right]
\left[
\begin{array}{c}
a_{1} \\ a_{2} \\ \vdots \\ a_{n-1} \\ a_{n} \\ k_{n-1} \\ k_{n-2} \\ \vdots \\ k_{2} \\ k_{1} \\ k_{0}
\end{array}
\right]
$$

$$
y = Y A
$$

$$
Y^T Y A = Y^T y
$$

$$
A = (Y^T Y)^{-1}Y^T y
$$

Then use the first $n$ elements of $A$ to form the transfer matrix of the integral equation:

$$
\bar{A} = 
\left[
\begin{array}{cccccc}
a_1 & a_2 & a_3 & \cdots & a_{n-1} & a_{n} \\
1 & 0 & 0 & \cdots & 0 & 0 \\
0 & 1 & 0 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 1 & 0
\end{array}
\right]
$$

Compute the eigenvalues of $\bar{A}$ to obtain the solution of the characteristic polynomial.

The eigenvalues $\lambda _n$ are then the values of the exponentials in:

$$
y = 
p_1 e^{\lambda _1 x} + 
p_2 e^{\lambda _2 x} + 
\cdots + 
p_{n-1} e^{\lambda _{n-1} x} + 
p_n e^{\lambda _n x}
$$

To obtain the missing $p_n$, calculate numerically the exponential terms $e^{\lambda _n}$ and solve the least squares problem for $[p_1, p_2, \cdots, p_{n-1}, p_n ]$ in

$$
y = [e^{\lambda _1 x}, e^{\lambda _2 x} , \cdots, e^{\lambda _{n-1} x}, e^{\lambda _n x}]
\left[
\begin{array}{c}
p_1 \\
p_2 \\
\vdots \\
p_{n-1} \\
p_n
\end{array}
\right]
$$

$$
y = X P
$$

$$
X^T X P = X^T y
$$

$$
P = (X^T X)^{-1}X^T y
$$

Now you have your $n$ exponential model parameters $p_n$ and $\lambda _n$.

---

# Example N = 4

## Import [PyArma](https://pyarma.sourceforge.io/)

In [1]:
from pyarma import *
from pyarma_utils import cumtrapz, join_horizontal

## Generate data to fit

In [2]:
dx = 0.02
x = regspace(dx, dx, 1.5)
y = 5*exp(0.5*x) + 4*exp(-3*x) + 2*exp(-2*x) - 3*exp(0.15*x)

## Compute integrals

In [3]:
iy1 = cumtrapz(x, y)
iy2 = cumtrapz(x, iy1)
iy3 = cumtrapz(x, iy2)
iy4 = cumtrapz(x, iy3)

## Compute exponentials lambdas

In [4]:
Y = join_horizontal(iy1, iy2, iy3, iy4, pow(x, 3), pow(x, 2), x, ones(size(x)))
A = pinv(Y)*y;
lambdas = eig_gen(mat([
    [A[0], A[1], A[2], A[3]],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0]
]))[0]
lambdas = real(lambdas)
lambdas.print("lambdas = ")

lambdas = 
  -2.9991
  -1.9997
   0.5000
   0.1500


## Compute exponentials multipliers

In [5]:
X = join_horizontal(exp(lambdas[0]*x), exp(lambdas[1]*x), exp(lambdas[2]*x), exp(lambdas[3]*x))
P = pinv(X)*y;
P.print("P = ")

P = 
   4.0042
   1.9955
   4.9998
  -2.9996
